In [1]:
from dash import Dash, html, dcc, callback, Output, Input,State
import plotly.express as px
import pandas as pd

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders.pdf import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredPowerPointLoader
from langchain.document_loaders.csv_loader import CSVLoader

embedding = OpenAIEmbeddings()

In [ ]:
app = Dash(__name__)
text_input_style = {'justify-content': 'center', 'align-items': 'center'}
app.layout = html.Div([
    html.H1(children='AskDoc', style={'textAlign':'center'}),
    html.Br(),
    html.H3(children='Enter File Location', style=text_input_style),
    html.Br(),
    dcc.Input(id="Input1", type="text", placeholder="", style={'marginRight':'10px','textAlign':'center','width': '100%'}),
    html.Br(),
    html.Br(),
    html.H3(children='Enter the query', style={'justify-content': 'center', 'align-items': 'center'}),
    dcc.Textarea(
        id='Query',
        value='Ask any question',
        style={'width': '100%', 'height': 300},
    ),
    html.Br(),
    html.Button('submit', id='submit-val', n_clicks=0),
    html.Br(),
    html.Div(id='textarea-example-output', style={'whiteSpace': 'pre-line'})
])

@app.callback(
    Output('textarea-example-output', 'children'),
    Input('submit-val','n_clicks'),
    State('Query', 'value'),
    State('Input1','value')
    )
def query_doc(n_clicks,value1,value2):
    if(n_clicks>0):
        if(".pdf" in value2):
            loader = PyPDFLoader(value2)
        elif(".txt" in value2):
            loader = TextLoader(value2)
        elif(".docx" in value2):
            loader = Docx2txtLoader(value2)
        elif(".pptx" in value2):
            loader = UnstructuredPowerPointLoader(value2,mode='elements')
        elif(".csv" in value2):
            loader = CSVLoader(value2)
        else:
            return "Unsupported file type"
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        texts = text_splitter.split_documents(documents)
        vectordb = Chroma.from_documents(documents=texts, embedding=embedding)
        qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectordb)
        query = value1
        result=qa.run(query)
        return result

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
[2023-05-08 00:45:38,520] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\ProgramData\anaconda3\lib\site-packages\dash\dash.py", line 1283, in dispatch
    ctx.run(
  File "C:\ProgramData\anaconda3\lib\site-packages\dash\_callback.py", line 450, in add_context
    output_value = func(*func_args, **func_